In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [7]:
data = pd.read_csv("IMDB Dataset.csv", escapechar='\\')
print(data.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [8]:
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords

english_stops = set(stopwords.words('english'))
print(list(english_stops)[:20])  # check first 20 stopwords

['o', 'should', 'so', "couldn't", 'there', "i'll", 'by', 'most', 'nor', 'a', 'didn', 'will', 'were', "hadn't", 'just', 'to', 'hers', 'yours', 'having', "you'll"]


In [12]:

def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


/tmp/ipython-input-3395383259.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
45267    [my, husband, i, went, see, movie, horror, mov...
37931    [fascinating, look, behind, scenes, really, go...
22796    [this, flick, reminds, really, bad, science, f...
20008    [apparently, movie, based, true, story, i, sur...
22728    [that, sound, stan, ollie, spinning, graves, i...
                               ...                        
11671    [after, watching, i, lost, little, respect, ch...
14361    [i, looking, forward, movie, trustworthy, acto...
21999    [wynorski, films, always, excreble, this, anot...
37295    [gilles, mamouni, playing, audience, story, ma...
39037    [you, thought, traumschiff, surprise, german, ...
Name: review, Length: 40000, dtype: object 

36574    [first, film, divided, three, segments, a, pro...
7537     [former, private, eye, turned, security, guard...
22559    [george, c, scott, gives, finest, funniest, wo...
29721    [i, say, much, this, director, raw, images, th...
20142    [wow, finally, jim, carrey, returned, died, th...
 

In [14]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [15]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  220   513     1 ...     0     0     0]
 [ 1343    78   430 ...     0     0     0]
 [    8   400  1689 ...     0     0     0]
 ...
 [ 8181    35   122 ...     0     0     0]
 [22313 46043   299 ...     0     0     0]
 [  101   103 92141 ...     0     0     0]] 

Encoded X Test
 [[   23     4  6192 ...     0     0     0]
 [ 1044  1909   648 ...     0     0     0]
 [  628   876   970 ...     0     0     0]
 ...
 [ 1921 17111    53 ...     0     0     0]
 [   38  3302   391 ...     0     0     0]
 [    1    42   242 ...    20   713  5223]] 

Maximum review length:  131


In [17]:
model = Sequential()
model.add(Embedding(input_dim=total_words,
                    output_dim=EMBED_DIM,
                    input_shape=(max_length,)))  # 👈 instead of input_length
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.build(input_shape=(None, max_length))  # 👈 force build before summary
print(model.summary())


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 131, 32)        │     2,948,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,973,441 (11.34 MB)

 Trainable params: 2,973,441 (11.34 MB)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [19]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.5459 - loss: 0.6771
Epoch 1: accuracy improved from -inf to 0.58745, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 60s 182ms/step - accuracy: 0.5460 - loss: 0.6770
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.5876 - loss: 0.6599
Epoch 2: accuracy improved from 0.58745 to 0.60725, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 55s 177ms/step - accuracy: 0.5876 - loss: 0.6599
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.6402 - loss: 0.6294
Epoch 3: accuracy improved from 0.60725 to 0.65610, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 179ms/step - accuracy: 0.6403 - loss: 0.6294
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.6429 - loss: 0.6265
Epoch 4: accuracy improved from 0.65610 to 0.66212, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 182ms/step - accuracy: 0.6429 - loss: 0.6264
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.8157 - loss: 0.4528
Epoch 5: accuracy improved from 0.66212 to 0.78123, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - accuracy: 0.8156 - loss: 0.4529


In [21]:
y_pred_prob = model.predict(x_test, batch_size=128)

# Convert probabilities to 0/1
y_pred = (y_pred_prob > 0.5).astype("int32")

# Count accuracy manually
true = np.sum(y_pred.flatten() == y_test)

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format(true / len(y_pred) * 100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step
Correct Prediction: 7954
Wrong Prediction: 2046
Accuracy: 79.54%


In [23]:
from tensorflow.keras.models import load_model

loaded_model = load_model("models/LSTM.h5")

y_pred = (loaded_model.predict(x_test) > 0.5).astype("int32")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step


In [25]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [27]:
loaded_model.compile(optimizer="adam",
                     loss="binary_crossentropy",
                     metrics=["accuracy"])

loss, acc = loaded_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7958 - loss: 0.4967
Test Accuracy: 0.80


In [29]:
import re

# Example review
review = "This movie was AMAZING!!! <br /> But some parts were boring..."

# Remove everything except letters and spaces
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

# Tokenize and remove stopwords
words = review.split(' ')
filtered = [w for w in words if w.lower() not in english_stops and w != '']  # also check lowercase + remove empty strings
filtered = ' '.join(filtered)
filtered = [filtered.lower()]  # put inside list (for tokenizer later)

print('Filtered: ', filtered)


Cleaned:  This movie was AMAZING br  But some parts were boring
Filtered:  ['movie amazing br parts boring']


In [30]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[    3   398 35348   426   259     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0]]


In [31]:
result = loaded_model.predict(tokenize_words)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
[[0.74706113]]


In [32]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
